<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/ml/Smartapi_duckdb_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyotp --q
!pip install smartapi-python==1.4.1 --q
!pip install logzero --q

In [4]:
# Package imports
from SmartApi import SmartConnect  # or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger
import time
import os
import urllib
import json
import pandas as pd
import datetime as dt

# API Credentials
api_key = 'xOHnB7MG'
username = 'M55123447'
pwd = '1471'
smartApi = SmartConnect(api_key)

# Token Authentication
try:
    token = "GJZACUQI2TTAIBHBA34XNFJURQ"
    totp = pyotp.TOTP(token).now()
except Exception as e:
    logger.error("Invalid Token: The provided token is not valid.")
    raise e

correlation_id = "abcde"
data = smartApi.generateSession(username, pwd, totp)

if not data.get('status', True):
    logger.error("Login failed: ", data)
else:
    authToken = data['data']['jwtToken']
    refreshToken = data['data']['refreshToken']
    feedToken = smartApi.getfeedToken()
    res = smartApi.getProfile(refreshToken)
    smartApi.generateToken(refreshToken)

# Fetch Stock List
nifty50_url = "https://raw.githubusercontent.com/anirbanghoshsbi/data/main/ind_nifty50list.csv"
stocks = pd.read_csv(nifty50_url)
nifty_50 = stocks.Symbol.tolist()

# Fetch Instrument Data
instrument_url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
response = urllib.request.urlopen(instrument_url)
instrument_list = json.loads(response.read())

# Token Lookup Functions
def token_lookup(ticker, instrument_list, exchange="NSE"):
    for instrument in instrument_list:
        if instrument["name"] == ticker and instrument["exch_seg"] == exchange and instrument["symbol"].split('-')[-1] == "EQ":
            return instrument["token"]
    return None  # Return None if not found

def hist_data(smartApi, tickers, interval, instrument_list, exchange="NSE"):
    hist_data_tickers = {}

    from_date = dt.datetime(2025, 5, 25).strftime('%Y-%m-%d %H:%M')  # Start from 2007
    to_date = dt.datetime.today().strftime('%Y-%m-%d %H:%M')  # Until today

    for ticker in tickers:
        token = token_lookup(ticker, instrument_list)
        if token is None:
            logger.error(f"Token not found for {ticker}")
            continue  # Skip if token not found

        params = {
            "exchange": exchange,
            "symboltoken": token,
            "interval": interval,
            "fromdate": from_date,
            "todate": to_date
        }
        print(f"Downloading data for {ticker}...")

        try:
            hist_data = smartApi.getCandleData(params)
            time.sleep(1.5)  # Avoid hitting rate limits

            df_data = pd.DataFrame(hist_data["data"], columns=["date", "open", "high", "low", "close", "volume"])
            df_data.set_index("date", inplace=True)
            df_data.index = pd.to_datetime(df_data.index).tz_localize(None)

            hist_data_tickers[ticker] = df_data
        except Exception as e:
            logger.error(f"Failed to fetch {ticker}: {e}")

    # Combine data for all tickers
    final_data = pd.concat(hist_data_tickers, axis=1)
    return final_data

# Download Nifty 50 Stocks Daily Data from 2007 to Present
candle_data = hist_data(smartApi, nifty_50, "ONE_DAY", instrument_list)

# Display Data
print(candle_data.columns)
print(candle_data.head())
print(candle_data.tail())

# Moving Average Calculation
def calculate_percentage_above_moving_average(data, periods):
    ma = data.ewm(span=periods, min_periods=periods).mean()
    above_ma = data > ma
    percentage = (above_ma.sum(axis=1) / 50) * 100
    return percentage


MultiIndex([(  'ADANIENT',   'open'),
            (  'ADANIENT',   'high'),
            (  'ADANIENT',    'low'),
            (  'ADANIENT',  'close'),
            (  'ADANIENT', 'volume'),
            ('ADANIPORTS',   'open'),
            ('ADANIPORTS',   'high'),
            ('ADANIPORTS',    'low'),
            ('ADANIPORTS',  'close'),
            ('ADANIPORTS', 'volume'),
            ...
            ('ULTRACEMCO',   'open'),
            ('ULTRACEMCO',   'high'),
            ('ULTRACEMCO',    'low'),
            ('ULTRACEMCO',  'close'),
            ('ULTRACEMCO', 'volume'),
            (     'WIPRO',   'open'),
            (     'WIPRO',   'high'),
            (     'WIPRO',    'low'),
            (     'WIPRO',  'close'),
            (     'WIPRO', 'volume')],
           length=250)
           ADANIENT                                 ADANIPORTS          \
               open    high     low   close  volume       open    high   
date                                                

In [7]:
candle_data.tail()


ADANIENT                                 ADANIPORTS          \
               open    high     low   close  volume       open    high   
date                                                                     
2025-07-01   2624.9  2645.0  2603.0  2621.8  729197     1450.3  1459.8   
2025-07-02   2634.0  2646.0  2613.6  2630.2  621409     1458.0  1458.0   
2025-07-03   2634.0  2644.0  2605.1  2611.2  502724     1444.0  1451.1   
2025-07-04   2617.0  2617.0  2582.2  2599.1  370305     1432.0  1439.9   
2025-07-07   2602.0  2605.4  2572.0  2576.1  412159     1424.4  1437.8   

                                     ... ULTRACEMCO                    \
               low   close   volume  ...       open     high      low   
date                                 ...                                
2025-07-01  1442.0  1447.0  2134736  ...    12105.0  12225.0  12060.0   
2025-07-02  1440.0  1443.9  1219600  ...    12210.0  12527.0  12170.0   
2025-07-03  1426.7  1431.1   978007  ...    12436.0  12575.0  12376.0   
2025-07-04  1416.6  1424.4   608462  ...    12400.0  12542.0  12341.0   
2025-07-07  1417.0  1435.6   764520  ...    12476.0  12500.0  12306.0   

                              WIPRO                                   
              close  volume    open    high     low   close   volume  
date                                                                  
2025-07-01  12210.0  207437  265.50  268.35  263.70  264.55  3886095  
2025-07-02  12436.0  445829  265.00  268.70  264.45  267.00  8750887  
2025-07-03  12398.0  275755  268.40  270.85  266.45  267.05  9357962  
2025-07-04  12506.0  191516  268.30  271.40  267.50  270.05  7141779  
2025-07-07  12345.0  149939  270.05  270.55  267.30  267.75  4849218  

[5 rows x 250 columns]

In [ ]:
import duckdb
    if store:
        long = (
            wide
            .stack(level=0)                          # -> index:(date, field) / columns:ticker
            .unstack(level=1)                        # -> index:date, columns:(field) / rows:multi-indexed by ticker
            .stack(level=0)
            .reset_index()
            .rename(columns={"level_1": "symbol"})
            [["date", "symbol", "open", "high", "low", "close", "volume"]]
            .dropna()
        )

        con = duckdb.connect(db_path)
        # create table once with a unique key on (date,symbol)
        con.execute(f"""
            CREATE TABLE IF NOT EXISTS {table} (
                date    TIMESTAMP,
                symbol  VARCHAR,
                open    DOUBLE,
                high    DOUBLE,
                low     DOUBLE,
                close   DOUBLE,
                volume  BIGINT,
                PRIMARY KEY (date, symbol)
            );
        """)
        # register Pandas DF and upsert
        con.register("tmp_df", long)
        con.execute(f"""
            INSERT OR REPLACE INTO {table}
            SELECT * FROM tmp_df;
        """)
        con.close()

    return wide

/tmp/ipython-input-4-2017986453.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  candle_data.to_sql("candle_table", conn, if_exists="append", index=True)


✅ Data saved successfully to candle_data.duckdb


In [ ]:
# Reconnect and load data from DuckDB
conn = duckdb.connect("candle_data.duckdb")

# Read data into a Pandas DataFrame
candle_tail = conn.execute("SELECT * FROM candle_table").df()

# Close connection
conn.close()

# Display first few rows
print(candle_tail.shape)


(11001, 51)


In [ ]:
if not data.get('status', True):
    logger.error("Login failed: ", data)
else:
    authToken = data['data']['jwtToken']
    refreshToken = data['data']['refreshToken']
    feedToken = smartApi.getfeedToken()
    res = smartApi.getProfile(refreshToken)
    smartApi.generateToken(refreshToken)

# Token for Nifty 50 Index (already provided)
nifty_50_token = "99926000"

# Function to fetch historical data for Nifty 50 index
def fetch_nifty50_data(smartApi, token, interval="ONE_DAY", exchange="NSE"):
    """Fetches historical daily data for the Nifty 50 index from 2007 to today."""

    from_date = dt.datetime(2025,5,25).strftime('%Y-%m-%d %H:%M')  # Start date
    to_date = dt.datetime.today().strftime('%Y-%m-%d %H:%M')  # End date

    params = {
        "exchange": exchange,
        "symboltoken": token,
        "interval": interval,
        "fromdate": from_date,
        "todate": to_date
    }

    print(f"Downloading Nifty 50 Index data from {from_date} to {to_date}...")

    try:
        hist_data = smartApi.getCandleData(params)
        time.sleep(1.5)  # Avoid hitting API rate limits

        df_data = pd.DataFrame(hist_data["data"], columns=["date", "open", "high", "low", "close", "volume"])
        df_data.set_index("date", inplace=True)
        df_data.index = pd.to_datetime(df_data.index).tz_localize(None)

        print("✅ Data downloaded successfully!")
        return df_data
    except Exception as e:
        logger.error(f"Failed to fetch Nifty 50 data: {e}")
        return None

# Fetch and store Nifty 50 index data
nifty50_data = fetch_nifty50_data(smartApi, nifty_50_token)

# Display Data
if nifty50_data is not None:
    print(nifty50_data.head())
    print(nifty50_data.tail())

    # Save data locally as CSV
    nifty50_data.to_csv("nifty50_data.csv")
    print("✅ Data saved as nifty50_data.csv")

    # Save to DuckDB for efficient querying
    import duckdb
    conn = duckdb.connect("nifty50_data.duckdb")
    nifty50_data.to_sql("nifty50_table", conn, if_exists="append", index=True)
    conn.close()
    print("✅ Data saved in DuckDB (nifty50_data.duckdb)")


✅ Data downloaded successfully!
                open      high       low     close  volume
date                                                      
2025-05-26  24919.35  25079.20  24900.50  25001.15       0
2025-05-27  24956.65  25062.90  24704.10  24826.20       0
2025-05-28  24832.50  24864.25  24737.05  24752.45       0
2025-05-29  24825.10  24892.60  24677.30  24833.60       0
2025-05-30  24812.60  24863.95  24717.40  24750.70       0
                open      high       low    close  volume
date                                                     
2025-07-01  25551.35  25593.40  25501.80  25541.8       0
2025-07-02  25588.30  25608.10  25378.75  25453.4       0
2025-07-03  25505.10  25587.50  25384.35  25405.3       0
2025-07-04  25428.85  25470.25  25331.65  25461.0       0
2025-07-07  25450.45  25489.80  25407.25  25426.9       0
✅ Data saved as nifty50_data.csv


/tmp/ipython-input-6-1996877820.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nifty50_data.to_sql("nifty50_table", conn, if_exists="append", index=True)


✅ Data saved in DuckDB (nifty50_data.duckdb)


In [ ]:
# Reconnect and load data from DuckDB
conn = duckdb.connect("nifty50_data.duckdb")

# Read data into a Pandas DataFrame
candle_tail = conn.execute("SELECT * FROM nifty50_table").df()

# Close connection
conn.close()

# Display first few rows
print(candle_tail.shape)


(5021, 6)


In [ ]:
candle_tail.tail(3)

,date,open,high,low,close,volume
5018,2025-07-03,25505.099609,25587.500000,25384.349609,25405.300781,0
5019,2025-07-04,25428.849609,25470.250000,25331.650391,25461.000000,0
5020,2025-07-07,25450.449219,25489.800781,25407.250000,25426.900391,0


In [ ]:
# Reconnect and load data from DuckDB
conn = duckdb.connect("candle_data.duckdb")

# Read data into a Pandas DataFrame
candle_tail = conn.execute("SELECT * FROM candle_table").df()

# Close connection
conn.close()

# Display first few rows
#print(candle_tail)


In [ ]:
candle_tail.tail(55)

,date,ADANIENT,ADANIPORTS,APOLLOHOSP,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,BPCL,...,SUNPHARMA,TCS,TATACONSUM,TATAMOTORS,TATASTEEL,TECHM,TITAN,UPL,ULTRACEMCO,WIPRO
10946,2025-06-03,2470.899902,1432.300049,6812.5,2255.699951,1181.500000,8563.0,897.900024,1991.900024,309.75,...,1667.500000,3405.199951,1114.599976,703.849976,157.350006,1543.800049,NaN,635.349976,11035.0,246.220001
10947,2025-06-04,2489.399902,1436.199951,6855.0,2249.000000,1171.000000,8563.5,898.700012,1956.099976,311.55,...,1664.900024,3380.899902,1124.400024,709.150024,158.179993,1556.699951,NaN,642.549988,11033.0,246.860001
10948,2025-06-05,2504.399902,1456.699951,6876.0,2243.500000,1159.000000,8557.5,893.400024,1944.000000,309.8,...,1683.099976,3371.100098,1111.900024,710.150024,157.970001,1562.800049,NaN,646.900024,11159.0,247.940002
10949,2025-06-06,2534.199951,1471.699951,6942.5,2245.199951,1194.599976,8637.0,937.150024,1990.000000,316.7,...,1679.199951,3384.199951,1115.599976,711.000000,157.490005,1571.099976,NaN,642.849976,11253.0,248.589996
10950,2025-06-09,2582.100098,1467.099976,6932.0,2247.899902,1219.900024,8641.0,960.750000,2017.400024,320.4,...,1694.400024,3421.899902,1121.199951,717.799988,157.320007,1579.099976,NaN,639.099976,11260.0,251.289993
10951,2025-06-10,2613.100098,1473.800049,6918.5,2218.899902,1232.800049,8626.0,949.750000,1998.099976,320.1,...,1688.800049,3462.899902,1115.099976,732.250000,155.679993,1610.900024,NaN,639.599976,11391.0,254.869995
10952,2025-06-11,2581.199951,1456.400024,6927.5,2208.800049,1231.400024,8719.0,942.500000,2012.900024,333.85,...,1690.599976,3471.899902,1104.300049,736.400024,156.410004,1637.500000,NaN,635.849976,11464.0,258.920013
10953,2025-06-12,2543.699951,1445.800049,6996.5,2219.399902,1212.900024,8567.0,936.849976,2028.800049,318.65,...,1687.400024,3434.199951,1085.099976,715.349976,152.889999,1644.400024,NaN,632.400024,11323.0,259.609985
10954,2025-06-13,2507.899902,1405.000000,6994.5,2214.199951,1205.800049,8463.5,933.099976,2011.000000,312.3,...,1687.800049,3445.699951,1078.400024,712.049988,152.130005,1659.000000,NaN,632.500000,11224.0,260.209991
10955,2025-06-16,2544.000000,1400.599976,7114.0,2244.800049,1215.400024,8529.5,938.000000,2028.199951,316.4,...,1685.300049,3496.300049,1083.400024,686.650024,154.169998,1693.900024,NaN,643.750000,11495.0,262.890015
